<a href="https://colab.research.google.com/github/arthurezende/rankingFaculdadesMec/blob/main/Projeto_dados_MEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Projeto dados MEC

In [ ]:
# Instalar dependências
!pip install pandas openpyxl
import pandas as pd

In [ ]:
# Conectar com Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Função para carregar DataFrame e tratar erros, lendo apenas colunas específicas
def load_dataframe(filepath, filetype, columns, **kwargs):
    try:
        if filetype == 'csv':
            return pd.read_csv(filepath, usecols=columns, **kwargs)
        elif filetype == 'excel':
            return pd.read_excel(filepath, usecols=columns, **kwargs)
    except FileNotFoundError:
        print(f"Arquivo {filepath} não encontrado.")
        return None
    except Exception as e:
        print(f"Erro ao carregar o arquivo {filepath}: {e}")
        return None

# Selecionar colunas relevantes
colunas_ies = ['NU_ANO_CENSO', 'SG_UF_IES', 'NO_MUNICIPIO_IES', 'TP_ORGANIZACAO_ACADEMICA', 'TP_REDE', 'CO_IES', 'NO_IES']
colunas_cursos = ['NU_ANO_CENSO', 'SG_UF', 'NO_MUNICIPIO', 'TP_REDE', 'CO_IES', 'NO_CURSO', 'CO_CURSO', 'NO_CINE_ROTULO', 'NO_CINE_AREA_ESPECIFICA', 'TP_GRAU_ACADEMICO', 'IN_GRATUITO', 'TP_MODALIDADE_ENSINO', 'QT_VG_TOTAL', 'QT_INSCRITO_TOTAL']
colunas_emec_ies = ['Código IES', 'CI', 'Ano CI', 'CI-EaD', 'Ano CI-EaD', 'IGC', 'Ano IGC']
colunas_emec_cursos = ['Código IES', 'Código Curso', 'CC', 'Ano CC', 'CPC', 'Ano CPC', 'ENADE', 'Ano ENADE', 'IDD', 'Ano IDD', 'Situação', 'Vagas Autorizadas']

### LEGENDA DOS DADOS

## Dados da IES
# NU_ANO_CENSO: Ano do Censo da IES (Fonte: Microdados IES)
# SG_UF_IES: Sigla da Unidade Federativa da IES
# NO_MUNICIPIO_IES: Nome do município da IES
# TP_ORGANIZACAO_ACADEMICA: Tipo de organização acadêmica da IES
# TP_REDE: Tipo de rede da IES (pública ou privada)
# CO_IES: Código da IES
# NO_IES: Nome da IES

## Dados do curso
# SG_UF: Sigla da Unidade Federativa do Curso
# NO_MUNICIPIO: Nome do município do Curso
# NO_CURSO: Nome do curso
# CO_CURSO: Código do curso
# NO_CINE_ROTULO: Nome do CINE (Classificação Nacional de Educação Superior) - Rótulo
# NO_CINE_AREA_ESPECIFICA: Nome da área específica do CINE
# TP_GRAU_ACADEMICO: Tipo de grau acadêmico do curso
# IN_GRATUITO: Indicador de gratuidade do curso
# TP_MODALIDADE_ENSINO: Tipo de modalidade de ensino do curso
# QT_VG_TOTAL: Quantidade de vagas totais do curso
# QT_INSCRITO_TOTAL: Quantidade de inscritos totais no curso

## Dados E-MEC IES
# CI: Conceito Institucional
# CI-EaD: Conceito Institucional do Ensino a Distância
# IGC: Índice Geral de Cursos
# Ano CI, Ano CI-EaD, Ano IGC: Ano em que foi realizada a avaliação

## Dados E-MEC Curso
# CC: Conceito de Curso - nota de 1 a 5 atribuída após avaliação presencial do INEP/MEC
# Ano CC: Ano em que foi realizada a avaliação presencial que gerou o CC
# CPC: Conceito Preliminar de Curso - indicador de 1 a 5 que avalia qualidade dos cursos de graduação
# Ano CPC: Ano em que foi calculado o Conceito Preliminar de Curso
# ENADE: Nota de 1 a 5 do Exame Nacional de Desempenho dos Estudantes
# Ano ENADE: Ano em que o curso participou do ENADE
# IDD: Indicador de Diferença entre Desempenhos Observado e Esperado - mede valor agregado pelo curso
# Ano IDD: Ano em que foi calculado o indicador IDD
# Situação: Status atual do curso (Ativo, Suspenso, Extinto, etc.)
# Vagas Autorizadas: Número de vagas que o MEC autorizou para o curso

# Carregar os dados das bases, especificando as colunas a serem lidas
df_ies = load_dataframe('/content/drive/MyDrive/Projeto Ext Faculdade Exame/MICRODADOS_ED_SUP_IES_2023.CSV', 'csv', colunas_ies, sep=';', encoding='latin-1', low_memory=False)
if df_ies is None:
    exit()

df_cursos = load_dataframe('/content/drive/MyDrive/Projeto Ext Faculdade Exame/MICRODADOS_CADASTRO_CURSOS_2023.CSV', 'csv', colunas_cursos, sep=';', encoding='latin-1', low_memory=False)
if df_cursos is None:
    exit()

df_emec_ies = load_dataframe('/content/drive/MyDrive/Projeto Ext Faculdade Exame/consulta-publica-emec-ies-20241205.xlsx', 'excel', colunas_emec_ies)
if df_emec_ies is None:
    exit()

df_emec_cursos = load_dataframe('/content/drive/MyDrive/Projeto Ext Faculdade Exame/consulta-publica-emec-cursos-20241205.xlsx', 'excel', colunas_emec_cursos)
if df_emec_cursos is None:
    exit()

In [ ]:
# Renomear colunas
df_emec_ies = df_emec_ies.rename(columns={'Código IES': 'CO_IES'})
df_emec_cursos.rename(columns={'Código Curso': 'CO_CURSO'}, inplace=True)
df_emec_cursos.drop_duplicates(subset=['CO_CURSO'], inplace=True)


# Juntar os dataframes
df_ies_e_cursos = pd.merge(df_ies, df_cursos, on='CO_IES', how='inner')
df_merged = pd.merge(df_ies_e_cursos, df_emec_ies, on='CO_IES', how='inner')
df_merged = pd.merge(df_merged, df_emec_cursos, on='CO_CURSO', how='left')

# Remover colunas duplicadas (_x e _y)
colunas_originais = {}
for col in df_merged.columns:
    if '_x' in col or '_y' in col:
        col_original = col.replace('_x', '').replace('_y', '')
        if col_original in colunas_originais:
            # Se a coluna original já foi encontrada, remova a coluna atual
            df_merged.drop(columns=[col], inplace=True)
        else:
            # Se a coluna original ainda não foi encontrada, renomeie a coluna atual
            colunas_originais[col_original] = col
            df_merged.rename(columns={col: col_original}, inplace=True)

# Exportar para CSV
try:
    df_merged.to_csv('dados_combinados.csv', index=False, encoding='utf-8')
    print("Arquivo dados_combinados.csv criado com sucesso!")
except Exception as e:
    print(f"Erro ao exportar o arquivo: {e}")

# Exibir informações e realizar verificações
#print(df_merged.info())
#print(df_merged.describe())
#print(df_merged.head())

#print(f"O DataFrame df_merged possui {df_merged.shape[0]} linhas e {df_merged.shape[1]} colunas.")

# Filtrar e exibir cursos para um CO_IES específico
#cursos_co_ies = df_merged[df_merged['CO_IES'] == 322]
#print(cursos_co_ies)

#Imprimir colunas para verificação
#print("Colunas df_cursos:", df_cursos.columns.tolist())
#print("\nColunas df_ies:", df_ies.columns.tolist())
#print("\nColunas df_emec_ies:", df_emec_ies.columns.tolist())
#print("\nColunas df_emec_cursos:", df_emec_cursos.columns.tolist())
#print("\nColunas df_merged:", df_merged.columns.tolist())

Arquivo dados_combinados.csv criado com sucesso!


In [ ]:
import os

# Embaralha o DataFrame
df_merged = df_merged.sample(frac=1, random_state=42).reset_index(drop=True)

# Conta a quantidade de linhas por UF_IES
uf_counts = df_merged['SG_UF_IES'].value_counts()

# Calcula a quantidade de linhas a serem amostradas por UF_IES
sample_size = 50000
uf_sample_sizes = (uf_counts / uf_counts.sum()) * sample_size

# Cria um novo DataFrame com as amostras
sampled_df = pd.DataFrame()
for uf, count in uf_sample_sizes.items():
    uf_df = df_merged[df_merged['SG_UF_IES'] == uf].sample(n=int(count), random_state=42)
    sampled_df = pd.concat([sampled_df, uf_df])

# Define o caminho do arquivo
filepath = '/content/drive/MyDrive/Projeto Ext Faculdade Exame/dados_reduzidos_50_mil_linhas-uf.csv'

# Exporta o DataFrame para CSV
try:
    sampled_df.to_csv(filepath, index=False, encoding='utf-8')
    print(f"Arquivo '{filepath}' criado com sucesso!")

    # Informações sobre o arquivo
    file_size = os.path.getsize(filepath)
    print(f"Tamanho do arquivo: {file_size} bytes")

    # Lê o arquivo exportado
    df_exportado = pd.read_csv(filepath)
    num_linhas = len(df_exportado)
    print(f"Número de linhas no arquivo exportado: {num_linhas}")
    print("Primeiras linhas do arquivo exportado:")
    print(df_exportado.head())
    print(f"O arquivo possui {df_exportado.shape[1]} colunas.")
    print("\nEstatísticas do arquivo exportado:")
    print(df_exportado.describe())
except Exception as e:
    print(f"Erro ao exportar ou processar o arquivo: {e}")

In [ ]:
# Instalando Streamlit para gerar uma visualização web
!pip install -q streamlit
!pip install streamlit-aggrid
!npm install localtunnel

In [ ]:
# Imprimir o IP para usar no localtunnel
print("Password/Endpoint IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!streamlit run app.py & npx localtunnel --port 8501